# Import software libraries

In [1]:
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
- pandas 1.4.4
- sqlite3 3.31.1


# Load a CSV file as a `DataFrame`

In [2]:
complaints_data = pd.read_csv("/content/consumer_loan_complaints.csv")

# Preview the first three rows of the data

In [3]:
complaints_data.head(n = 3)

,user_id,Date received,Product,Issue,Consumer complaint narrative,State,ZIP code,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,44fefdad-7045-4be5-890e-12e84ae6fdc9,01/27/2016,Consumer Loan,Account terms and changes,NaN,AL,35180,Phone,01/27/2016,Closed with explanation,Yes,No,1760486
1,c49d5d60-909f-406b-b7ff-51143fcb650b,08/26/2014,Consumer Loan,Account terms and changes,NaN,NC,278XX,Phone,08/29/2014,Closed with non-monetary relief,Yes,No,1001740
2,9b2cd5d2-900e-4052-831f-6489f6d568af,08/22/2012,Consumer Loan,Account terms and changes,NaN,TN,37205,Referral,08/23/2012,Closed with non-monetary relief,Yes,No,140039


# Create a connection to the SQLite database

In [5]:
conn = sqlite3.connect("/content/user_data.db")
conn

# Read the `users` data

In [6]:
# Write a query that selects everything from the users table.
query = 'SELECT * FROM users'

In [7]:
# Read the query into a DataFrame.
users = pd.read_sql(query, conn)
# Preview the data.
users.head()

,user_id,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,58,management,married,tertiary,no,yes,no,None,261,1,-1,0,None,no,1998-08-23
1,bb92765a-08de-4963-b432-496524b39157,44,technician,single,secondary,no,yes,no,None,151,1,-1,0,None,no,2008-07-15
2,573de577-49ef-42b9-83da-d3cfb817b5c1,33,entrepreneur,married,secondary,no,yes,yes,None,76,1,-1,0,None,no,2002-06-04
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,47,blue-collar,married,None,no,yes,no,None,92,1,-1,0,None,no,1995-06-29
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,33,None,single,None,no,no,no,None,198,1,-1,0,None,no,1995-08-01


In [9]:
# Check the shape of the data.
users.shape

(45216, 16)

# Read the `device` data

In [10]:
query = "SELECT * FROM device"
device = pd.read_sql(query, conn)
device.head()

,user_id,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,mobile
1,bb92765a-08de-4963-b432-496524b39157,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,mobile


In [11]:
device.shape

(45117, 2)

# Read the `transactions` data

In [12]:
# Read the user transactions in the last 30 days. 
query = "SELECT * FROM transactions"
transactions = pd.read_sql(query, conn)
transactions.head()

,user_id,transaction_id,amount_usd
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,transaction_5180,1332
1,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,transaction_5607,726
2,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,transaction_6765,85
3,573de577-49ef-42b9-83da-d3cfb817b5c1,transaction_6170,1
4,573de577-49ef-42b9-83da-d3cfb817b5c1,transaction_6090,1


In [13]:
transactions.shape

(140034, 3)

# Aggregate the `transactions` data

In [14]:
# Aggregate data on the number of transactions and the total amount.
query = """SELECT user_id, COUNT(*) AS number_transactions, SUM(amount_usd) AS total_amount_usd
FROM transactions
GROUP BY user_id
"""
transactions_agg = pd.read_sql(query, conn)
transactions_agg.head()

,user_id,number_transactions,total_amount_usd
0,0001570d-8aed-465e-b547-8981651084ed,3,792
1,000548ed-aa18-4eef-b8ed-68a9126e33ab,2,1044
2,00069959-4d55-460e-bb76-ae13ddbd80a6,5,0
3,000bab00-aec4-4ee2-81a6-1f897c38726b,19,0
4,000cbac8-212f-46fb-b58f-861dada34284,2,399


In [15]:
transactions.shape

(140034, 3)

# Merge the `device` table with the `users` table

In [17]:
# Do a left join, as all users in the users table are of interest.
query = """
SELECT left_table.*,
right_table.device
FROM users AS left_table
LEFT JOIN device AS right_table
ON left_table.user_id = right_table.user_id
"""
users_w_device = pd.read_sql(query, conn)

In [18]:
users_w_device.head(n = 3)

,user_id,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,58,management,married,tertiary,no,yes,no,None,261,1,-1,0,None,no,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,44,technician,single,secondary,no,yes,no,None,151,1,-1,0,None,no,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,33,entrepreneur,married,secondary,no,yes,yes,None,76,1,-1,0,None,no,2002-06-04,mobile


# Close the database connection

In [19]:
conn.close()

# Merge `users_w_device` with `transactions_agg`

In [20]:
# Do a right join so users won't be lost.
users_w_devices_and_transactions = \
transactions_agg.merge(users_w_device,
                       on = "user_id", how = "right")
users_w_devices_and_transactions.head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,no,yes,no,None,261,1,-1,0,None,no,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,no,yes,no,None,151,1,-1,0,None,no,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,no,yes,yes,None,76,1,-1,0,None,no,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,None,no,yes,no,None,92,1,-1,0,None,no,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,None,single,None,no,no,no,None,198,1,-1,0,None,no,1995-08-01,mobile


In [21]:
# Make sure number of rows is equal to users_w_devices table.
users_w_devices_and_transactions.shape

(45216, 19)

# Identify data where `age` is greater than 150

In [22]:
users_w_devices_and_transactions[users_w_devices_and_transactions.age > 150]

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
7228,44fefdad-7045-4be5-890e-12e84ae6fdc9,NaN,NaN,178,blue-collar,married,primary,no,yes,no,None,691,1,-1,0,None,no,1997-04-28,desktop
10318,9b2cd5d2-900e-4052-831f-6489f6d568af,2.0,3165.0,891,management,married,tertiary,no,yes,no,None,278,2,-1,0,None,no,2003-10-03,mobile


# Drop incorrect data

In [23]:
users_cleaned = \
users_w_devices_and_transactions[users_w_devices_and_transactions.age < 150]
users_cleaned.shape

(45214, 19)

# Identify more potentially erroneous data

In [24]:
# Compare age to device.
pd.crosstab(users_cleaned["age"],
            users_cleaned["device"])

device,desktop,mobile,tablet
age,,,
18,5,6,1
19,10,22,3
20,11,33,6
21,16,44,19
22,30,87,11
...,...,...,...
90,1,1,0
92,1,1,0
93,0,2,0


# Identify data types that need correcting

In [25]:
users_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45214 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              45214 non-null  object 
 1   number_transactions  35215 non-null  float64
 2   total_amount_usd     35215 non-null  float64
 3   age                  45214 non-null  int64  
 4   job                  44926 non-null  object 
 5   marital              45214 non-null  object 
 6   education            43357 non-null  object 
 7   default              45214 non-null  object 
 8   housing              45214 non-null  object 
 9   loan                 45214 non-null  object 
 10  contact              32196 non-null  object 
 11  duration             45214 non-null  int64  
 12  campaign             45214 non-null  int64  
 13  pdays                45214 non-null  int64  
 14  previous             45214 non-null  int64  
 15  poutcome             8255 non-null  

In [26]:
users_cleaned.default.value_counts()

no     44398
yes      816
Name: default, dtype: int64

# Convert the relevant variables to a Boolean type

In [27]:
users_cleaned_1 = users_cleaned.copy()

users_cleaned_1.default = \
users_cleaned_1.default.map(dict(yes = 1, no = 0)).astype(bool)

users_cleaned_1.default.value_counts()

False    44398
True       816
Name: default, dtype: int64

In [28]:
# Do the same for the other Boolean variables.
bool_vars = ["housing", "loan", "term_deposit"]

for var in bool_vars:
  users_cleaned_1[var] = \
  users_cleaned_1[var].map(dict(yes = 1, no = 0)).astype(bool)

  print(f"Converted {var} to Boolean.")

Converted housing to Boolean.
Converted loan to Boolean.
Converted term_deposit to Boolean.


In [29]:
users_cleaned_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45214 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              45214 non-null  object 
 1   number_transactions  35215 non-null  float64
 2   total_amount_usd     35215 non-null  float64
 3   age                  45214 non-null  int64  
 4   job                  44926 non-null  object 
 5   marital              45214 non-null  object 
 6   education            43357 non-null  object 
 7   default              45214 non-null  bool   
 8   housing              45214 non-null  bool   
 9   loan                 45214 non-null  bool   
 10  contact              32196 non-null  object 
 11  duration             45214 non-null  int64  
 12  campaign             45214 non-null  int64  
 13  pdays                45214 non-null  int64  
 14  previous             45214 non-null  int64  
 15  poutcome             8255 non-null  

# Convert `date_joined` to a datetime format

In [30]:
# Work with a new object.
users_cleaned_2 = users_cleaned_1.copy()

users_cleaned_2["date_joined"] = \
pd.to_datetime(users_cleaned_2["date_joined"],
               format = "%Y-%m-%d")

In [31]:
users_cleaned_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45214 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              45214 non-null  object        
 1   number_transactions  35215 non-null  float64       
 2   total_amount_usd     35215 non-null  float64       
 3   age                  45214 non-null  int64         
 4   job                  44926 non-null  object        
 5   marital              45214 non-null  object        
 6   education            43357 non-null  object        
 7   default              45214 non-null  bool          
 8   housing              45214 non-null  bool          
 9   loan                 45214 non-null  bool          
 10  contact              32196 non-null  object        
 11  duration             45214 non-null  int64         
 12  campaign             45214 non-null  int64         
 13  pdays                45214 non-

# Identify all duplicated data

In [32]:
duplicated_data = \
users_cleaned_2[users_cleaned_2.duplicated(keep = False)]

print("Number of rows with duplicated data: ",
      duplicated_data.shape[0])

Number of rows with duplicated data:  10


In [33]:
duplicated_data.head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
15456,cba59442-af3c-41d7-a39c-0f9bffba0660,2.0,1218.0,57,management,married,tertiary,True,True,False,cellular,317,6,-1,0,None,False,1993-02-27,desktop
22005,1e826721-b38c-41c2-88f4-4c28b335b1e6,4.0,159.0,31,technician,single,secondary,False,False,False,cellular,129,1,-1,0,None,False,1990-05-10,mobile
35413,a2fb8264-d55a-437b-a8e7-9ec4116b76f4,2.0,676.0,34,management,married,tertiary,False,False,False,cellular,156,1,177,1,success,False,1989-09-02,mobile
35620,f49ac08f-b872-4d57-ac82-9b8a9144020d,4.0,117.0,38,blue-collar,married,secondary,False,True,False,cellular,54,1,337,1,failure,False,2010-06-11,mobile
36292,ae3b92a2-cad8-434f-8037-9815e2228839,2.0,426.0,43,admin.,single,secondary,False,True,False,cellular,76,2,304,2,failure,False,1992-10-29,desktop


# Remove the duplicated data

In [34]:
users_cleaned_final = \
users_cleaned_2[~users_cleaned_2.duplicated()]

users_cleaned_final[users_cleaned_final["user_id"] == \
                    'cba59442-af3c-41d7-a39c-0f9bffba0660']

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
15456,cba59442-af3c-41d7-a39c-0f9bffba0660,2.0,1218.0,57,management,married,tertiary,True,True,False,cellular,317,6,-1,0,None,False,1993-02-27,desktop


In [35]:
users_cleaned_final.shape

(45209, 19)

# Load data into an SQL database

In [38]:
conn = sqlite3.connect("users_data_cleaned.db")
users_cleaned_final.to_sql("users_cleaned_final", conn, if_exists = "replace", index = False)

45209

# Confirm that data was loaded into the database

In [39]:
query = "SELECT * FROM users_cleaned_final"
pd.read_sql(query, conn).head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,0,1,0,None,261,1,-1,0,None,0,1998-08-23 00:00:00,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,0,1,0,None,151,1,-1,0,None,0,2008-07-15 00:00:00,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,0,1,1,None,76,1,-1,0,None,0,2002-06-04 00:00:00,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,None,0,1,0,None,92,1,-1,0,None,0,1995-06-29 00:00:00,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,None,single,None,0,0,0,None,198,1,-1,0,None,0,1995-08-01 00:00:00,mobile


# Close the database connection

In [40]:
conn.close()

# Write the `DataFrame` as a pickle file

In [43]:
users_cleaned_final.to_pickle('users_data_cleaned.pickle')

# Confirm that the data was written to the pickle file

In [44]:
pd.read_pickle("users_data_cleaned.pickle").head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,None,261,1,-1,0,None,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,False,True,False,None,151,1,-1,0,None,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,None,76,1,-1,0,None,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,None,False,True,False,None,92,1,-1,0,None,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,None,single,None,False,False,False,None,198,1,-1,0,None,False,1995-08-01,mobile


In [45]:
pd.read_pickle("users_data_cleaned.pickle").info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45209 entries, 0 to 45210
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              45209 non-null  object        
 1   number_transactions  35210 non-null  float64       
 2   total_amount_usd     35210 non-null  float64       
 3   age                  45209 non-null  int64         
 4   job                  44921 non-null  object        
 5   marital              45209 non-null  object        
 6   education            43352 non-null  object        
 7   default              45209 non-null  bool          
 8   housing              45209 non-null  bool          
 9   loan                 45209 non-null  bool          
 10  contact              32191 non-null  object        
 11  duration             45209 non-null  int64         
 12  campaign             45209 non-null  int64         
 13  pdays                45209 non-

# Write the data to a CSV file

In [46]:
users_cleaned_final.to_csv("users_data_cleaned.csv", index = False)

# Confirm that the data was written to a CSV file.

In [47]:
pd.read_csv("users_data_cleaned.csv").head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,NaN,261,1,-1,0,NaN,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,False,True,False,NaN,151,1,-1,0,NaN,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,NaN,76,1,-1,0,NaN,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,NaN,False,True,False,NaN,92,1,-1,0,NaN,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,NaN,single,NaN,False,False,False,NaN,198,1,-1,0,NaN,False,1995-08-01,mobile


In [49]:
pd.read_csv("users_data_cleaned.csv").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45209 entries, 0 to 45208
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              45209 non-null  object 
 1   number_transactions  35210 non-null  float64
 2   total_amount_usd     35210 non-null  float64
 3   age                  45209 non-null  int64  
 4   job                  44921 non-null  object 
 5   marital              45209 non-null  object 
 6   education            43352 non-null  object 
 7   default              45209 non-null  bool   
 8   housing              45209 non-null  bool   
 9   loan                 45209 non-null  bool   
 10  contact              32191 non-null  object 
 11  duration             45209 non-null  int64  
 12  campaign             45209 non-null  int64  
 13  pdays                45209 non-null  int64  
 14  previous             45209 non-null  int64  
 15  poutcome             8252 non-null  